In [1]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
os.environ["TORCH_USE_CUDA_DSA"] = "1"

In [2]:
lcat_text_path = "resources/images/hanbury/extracted_text.txt"#./resources/images/lightfootcat/images/extracted_text_tess2.txt"


def read_file(file_path):
    """Read the content of a file."""
    with open(file_path, "r") as file:
        return file.read()


lcat_text = read_file(lcat_text_path)

print("Hanbury text loaded successfully.")

Hanbury text loaded successfully.


In [27]:
len(lcat_text)

221854

In [3]:
import re

## Cleaning

#### Page numbers and Catalogue common headers

In [4]:
from lib.data_processing.text_processing import TextProcessor

tp = TextProcessor()

In [5]:
divisions = ["Dicotyledones", "Monocotyledones"]

clean_text = tp.preprocess_text(lcat_text, divisions[0])

print(clean_text[:5000])

DICOTYLEDONES.
I. POLYPETALAE. I.FLORAE

RANUNCULACEAE

.
1. Aconitum Napellus, L.
a. A portion of the plant in flower.
Hohneck, Vosges, 13 Aug., 1855. Pic de Sancy, Auvergne,
Sept., 1858.
b. Plant in flower, with root attached.
2. Aconitum Lycocotonum, L.
a. Plant in flower, with root attached.
Hohneck, Vosges, 13 Aug., 1855.
3. Actaea spicata, L.
a. Leafy stem with flowers and detached fruit. Cultivated at Clapham Common, 1867.
b. Two leafy stems with fruit, and detached fruit.
(1) Hohneck, Vosges, 13 Aug., 1855. (2) Auvergne, Sept.,
1855.
4. Anemone Hepatica, L.
a. Plant in flower, two specimens.
(1) Salzburg, 10 Sept., 1862. With root attached.
(2) Mont. Ventoux, near Avignon, 7 May, 1866.
THALAMIFLORA.
5. Anemone Pulsatilla, L.
a. Plant in flower, three specimens.
(1) Chalky hills, Hexton, near Hitchin, Herts., April, 1854;
R. Bentley. (2) Hitchin, 1857; B. Hanbury. (3)
with root attached, near Cambridge, 1859; T. W.
Beddome.
b. Plants with flower, fruit, and root.
Tonnerre, Franc

In [6]:
div_struct = tp.split_by_divisions(clean_text, divisions)

In [7]:
div_struct

{'DICOTYLEDONES': '\nI. POLYPETALAE. I.FLORAE\n\nRANUNCULACEAE\n\n.\n1. Aconitum Napellus, L.\na. A portion of the plant in flower.\nHohneck, Vosges, 13 Aug., 1855. Pic de Sancy, Auvergne,\nSept., 1858.\nb. Plant in flower, with root attached.\n2. Aconitum Lycocotonum, L.\na. Plant in flower, with root attached.\nHohneck, Vosges, 13 Aug., 1855.\n3. Actaea spicata, L.\na. Leafy stem with flowers and detached fruit. Cultivated at Clapham Common, 1867.\nb. Two leafy stems with fruit, and detached fruit.\n(1) Hohneck, Vosges, 13 Aug., 1855. (2) Auvergne, Sept.,\n1855.\n4. Anemone Hepatica, L.\na. Plant in flower, two specimens.\n(1) Salzburg, 10 Sept., 1862. With root attached.\n(2) Mont. Ventoux, near Avignon, 7 May, 1866.\nTHALAMIFLORA.\n5. Anemone Pulsatilla, L.\na. Plant in flower, three specimens.\n(1) Chalky hills, Hexton, near Hitchin, Herts., April, 1854;\nR. Bentley. (2) Hitchin, 1857; B. Hanbury. (3)\nwith root attached, near Cambridge, 1859; T. W.\nBeddome.\nb. Plants with flowe

In [8]:
struct = dict()
for current_div, div_content in div_struct.items():
    current_div = current_div.strip()
    #print(f"==> Processing {current_div}")


    family_split = tp.split_by_families(div_content)
    struct.setdefault(current_div, family_split)
        

In [9]:
sample = struct["DICOTYLEDONES"][0]

In [10]:
sample_text = sample['text']

In [11]:
print(sample_text)



.
1. Aconitum Napellus, L.
a. A portion of the plant in flower.
Hohneck, Vosges, 13 Aug., 1855. Pic de Sancy, Auvergne,
Sept., 1858.
b. Plant in flower, with root attached.
2. Aconitum Lycocotonum, L.
a. Plant in flower, with root attached.
Hohneck, Vosges, 13 Aug., 1855.
3. Actaea spicata, L.
a. Leafy stem with flowers and detached fruit. Cultivated at Clapham Common, 1867.
b. Two leafy stems with fruit, and detached fruit.
(1) Hohneck, Vosges, 13 Aug., 1855. (2) Auvergne, Sept.,
1855.
4. Anemone Hepatica, L.
a. Plant in flower, two specimens.
(1) Salzburg, 10 Sept., 1862. With root attached.
(2) Mont. Ventoux, near Avignon, 7 May, 1866.
THALAMIFLORA.
5. Anemone Pulsatilla, L.
a. Plant in flower, three specimens.
(1) Chalky hills, Hexton, near Hitchin, Herts., April, 1854;
R. Bentley. (2) Hitchin, 1857; B. Hanbury. (3)
with root attached, near Cambridge, 1859; T. W.
Beddome.
b. Plants with flower, fruit, and root.
Tonnerre, France, 27 April, 1866.
6. Clematis Caripensis, H.B.K. "Cre

In [12]:
prompt = "resources/prompts/hanbury_prompt.yaml"

from lib.model.ocr_model import OCRModel

ocr_model = OCRModel(prompt)

/mnt/apps/users/ikarunak/conda/envs/dd_lcat/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[0917 09:06.59 @file_utils.py:31]  INF  PyTorch version 2.8.0+cu129 available.
[0917 09:06.59 @file_utils.py:69]  INF  Disabling Tensorflow because USE_TORCH is set


2025-09-17 09:07:01,588 - lib.utils.promptLoader - INFO - Loading prompt from resources/prompts/hanbury_prompt.yaml
2025-09-17 09:07:01,620 - lib.utils.promptLoader - INFO - Loading prompt from ./resources/settings.yaml
2025-09-17 09:07:01,643 - lib.model.base_model - INFO - Loading model: mistral7b with batch size: 1, max tokens: 4096, temperature: 0.01
2025-09-17 09:07:05,144 - lib.model - INFO - Importing model: mistral7b
2025-09-17 09:07:05,147 - lib.model - INFO - Model imported successfully: mistral7b


In [13]:
def prettyPrint(message):
    """Print the message in a readable format."""
    print("\n" + "="*50)
    print(message)
    print("="*50 + "\n")

In [28]:

system_prompt = (
    "You are an expert in structuring botanical text.\n"
    "You will be provided with a text containing botanical information from a historical botanical catalogue.\n"
    "The text contains botanical information; species names, and other relevant details.\n"
    "This information denotes how each specimen is stored in the catalogue.\n"
    "Your task is to identify each distinct species record in the text and insert the marker '---New-Record---' between them.\n"
    "Do not perform any text cleaning or make any other changes to the original text.\n"
    "Only return the text with the added markers, without any additional comments or explanations.\n"
    "If context is provided, use it as an example but do not return it."
    "Few-Shot Examples:\n"
    "Input: 'Acer campestre L.\n1 folder. Acer campestre [TA]\n\nAcer pseudoplatanus L.\n2 folders. 07\nFolder 1. Acer Pseudo-Platanus\n[(G]. i. \"Maple. Bulls: [Bulstrode]\nPark\" [JL]\nFolder 2. Acer Pseudo-Platanus\n[TA].'\n"
    "Output: 'Acer campestre L.\n1 folder. Acer campestre [TA]\n\n---New-Record---\nAcer pseudoplatanus L.\n2 folders. 07\nFolder 1. Acer Pseudo-Platanus\n[(G]. i. \"Maple. Bulls: [Bulstrode]\nPark\" [JL]\nFolder 2. Acer Pseudo-Platanus\n[TA].'\n"
    "Input: '.\n\nAmaranthus lividus L., Flora Europaea\n1: 110 (1964)\n1 folder. Amaranthus Blitum [TA].\ni. Cites Ray\\'s Syn. 1957. ii. \"Blite\nAmaranth. Aug.! It is often found\non Dunghills in the neighbourhood\nof London. [ gather\\'d this on a\nDunghill at Fulham near London\"\nUL]. iii, \"Amaranthus Blitum.\nMonoec: 5. and:\" [JL]'\n"
    "Output: '.\n\n---New-Record---\nAmaranthus lividus L., Flora Europaea\n1: 110 (1964)\n1 folder. Amaranthus Blitum [TA].\ni. Cites Ray\\'s Syn. 1957. ii. \"Blite\nAmaranth. Aug.! It is often found\non Dunghills in the neighbourhood\nof London. [ gather\\'d this on a\nDunghill at Fulham near London\"\nUL]. iii, \"Amaranthus Blitum.\nMonoec: 5. and:\" [JL]'"
)

user_prompt = lambda text: (
    "By following the rules, format the following text with markers:\n\n"
    f"{text}\n"
)

conversation = lambda text: [{"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt(text)}]

# sample = family_chunks["Dicotyledones"][8][1][:1000]

# prettyPrint(f"Sample text for analysis:\n{sample}") 

# conversation = [{"role": "system", "content": system_prompt},
#                 {"role": "user", "content": user_prompt(sample)}]


# output = ocr_model.model(conversation=conversation)


# prettyPrint(f"Output from the model:\n{output[0]}")



In [34]:
from tqdm import tqdm

In [35]:
def add_new_records(ocr_model, text, context_window=600):

    splitter = "---New-Record---"
    records = []
    join_text = ""
    context = ""

    for i in tqdm(range(0, len(text), context_window), desc="Chunking..."):
        chunk = text[i:i+context_window]
        if i == 0:
            context = ""
            join_text = chunk
        else:
            context += splitter.join(records[-3:-1])
            join_text = records[-1] + chunk

        message = f"Context: \n{context}\n\n Text:\n{join_text}\n\n" if context else join_text
        current_conv = conversation(message)
        output = ocr_model.model(conversation=current_conv)
        #prettyPrint(output[0])
        records.extend(output[0].split(splitter))
    
    return records



In [36]:
records = add_new_records(ocr_model, sample_text, context_window=600)

Chunking...:   0%|          | 0/16 [00:00<?, ?it/s]The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Chunking...:   6%|▋         | 1/16 [00:30<07:35, 30.36s/it]The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Chunking...:  12%|█▎        | 2/16 [01:18<09:32, 40.91s/it]The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Chunking...:  19%|█▉        | 3/16 [02:20<10:58, 50.65s/it]The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Chunking...:  25%|██▌       | 4/16 [03:10<10:03, 50.28s/it]The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` t

In [37]:
print("---New-Record---".join(records))

.
1. Aconitum Napellus, L.
a. A portion of the plant in flower.
Hohneck, Vosges, 13 Aug., 1855. Pic de Sancy, Auvergne,
Sept., 1858.
b. Plant in flower, with root attached.
---New-Record---
2. Aconitum Lycocotonum, L.
a. Plant in flower, with root attached.
Hohneck, Vosges, 13 Aug., 1855.
---New-Record---
3. Actaea spicata, L.
a. Leafy stem with flowers and detached fruit. Cultivated at Clapham Common, 1867.
b. Two leafy stems with fruit, and detached fruit.
(1) Hohneck, Vosges, 13 Aug., 1855. (2) Auvergne, Sept.,
1855.
---New-Record---
4. Anemone Hepatica, L.
a. Plant in flower, two specimens.
(1) Salzburg, 10 Sept., 1862. With root attache---New-Record---2. Aconitum Lycocotonum, L.
a. Plant in flower, with root attached.
Hohneck, Vosges, 13 Aug., 1855.

---New-Record---
3. Actaea spicata, L.
a. Leafy stem with flowers and detached fruit. Cultivated at Clapham Common, 1867.
b. Two leafy stems with fruit, and detached fruit.
(1) Hohneck, Vosges, 13 Aug., 1855. (2) Auvergne, Sept.,
1855

In [15]:
sample_conv = conversation(sample_text[:600])

output = ocr_model.model(conversation=sample_conv)
print(output[0])

Loading model for [mistralai/Mistral-7B-Instruct-v0.3] to device [cuda]


[0917 09:08.18 @modeling.py:1004]  INF  We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).
Loading checkpoint shards: 100%|██████████| 3/3 [00:23<00:00,  7.90s/it]


Loading processor for [mistralai/Mistral-7B-Instruct-v0.3] to device [cuda:0]
2025-09-17 09:08:44,663 - lib.model.hf_models.hf_model - WARNING - Pad token is None. Setting pad token to eos token.


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


.
1. Aconitum Napellus, L.
a. A portion of the plant in flower.
Hohneck, Vosges, 13 Aug., 1855. Pic de Sancy, Auvergne,
Sept., 1858.
b. Plant in flower, with root attached.
---New-Record---
2. Aconitum Lycocotonum, L.
a. Plant in flower, with root attached.
Hohneck, Vosges, 13 Aug., 1855.
---New-Record---
3. Actaea spicata, L.
a. Leafy stem with flowers and detached fruit. Cultivated at Clapham Common, 1867.
b. Two leafy stems with fruit, and detached fruit.
(1) Hohneck, Vosges, 13 Aug., 1855. (2) Auvergne, Sept.,
1855.
---New-Record---
4. Anemone Hepatica, L.
a. Plant in flower, two specimens.
(1) Salzburg, 10 Sept., 1862. With root attache


In [17]:
output_split = output[0].split("--New-Record--")
context = "--New-Record--".join(output_split[:-1])
join_text = output_split[-1]
next_sample_conv = conversation(f"Context: \n{context}\n\n Text:\n{join_text+sample_text[600:1000]}\n\n")

output_next = ocr_model.model(conversation=next_sample_conv)
print(output_next[0])

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Output:
-
1. Aconitum Napellus, L.
a. A portion of the plant in flower.
Hohneck, Vosges, 13 Aug., 1855. Pic de Sancy, Auvergne,
Sept., 1858.
b. Plant in flower, with root attached.
---New-Record---
2. Aconitum Lycocotonum, L.
a. Plant in flower, with root attached.
Hohneck, Vosges, 13 Aug., 1855.
---New-Record---
3. Actaea spicata, L.
a. Leafy stem with flowers and detached fruit. Cultivated at Clapham Common, 1867.
b. Two leafy stems with fruit, and detached fruit.
(1) Hohneck, Vosges, 13 Aug., 1855. (2) Auvergne, Sept.,
1855.
-

---New-Record---
4. Anemone Hepatica, L.
a. Plant in flower, two specimens.
(1) Salzburg, 10 Sept., 1862. With root attached.
(2) Mont. Ventoux, near Avignon, 7 May, 1866.
THALAMIFLORA.
---New-Record---
5. Anemone Pulsatilla, L.
a. Plant in flower, three specimens.
(1) Chalky hills, Hexton, near Hitchin, Herts., April, 1854;
R. Bentley. (2) Hitchin, 1857; B. Hanbury. (3)
with root attached, near Cambridge, 1859; T. W.
Beddome.
b. Plants with flower, fruit, an

In [ ]:
from tqdm import tqdm
import torch

outputs = []
for batch in tqdm(batches, desc="Processing batches", unit="batch"):
    with torch.no_grad():
        conversations = [conversation(chunk) for chunk in batch]
        outputs.extend(ocr_model.model(conversation=conversations))
    del conversations
    torch.cuda.empty_cache()
    #context = cut_output[-context_size:] if len(cut_output) > context_size else cut_output


Processing batches:   0%|          | 0/11 [00:00<?, ?batch/s]

Loading model for [mistralai/Mistral-7B-Instruct-v0.3] to device [cuda]


Loading checkpoint shards: 100%|██████████| 3/3 [00:22<00:00,  7.49s/it]


Loading processor for [mistralai/Mistral-7B-Instruct-v0.3] to device [cuda:0]
2025-08-07 13:58:53,504 - lib.model.hf_models.hf_model - WARNING - Pad token is None. Setting pad token to eos token.


/mnt/apps/users/ikarunak/conda/envs/lcat/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.01` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Processing batches: 100%|██████████| 11/11 [57:32<00:00, 313.88s/batch]


In [35]:
(outputs[1])

'\'Amaranthus lividus L., Flora Europaea\n1: 110 (1964)\n1 folder. Amaranthus Blitum [TA].\ni. Cites Ray\'s Syn. 1957.\nii. "Blite Amaranth. Aug. It is often found on Dunghills in the neighbourhood of London. [gather\'d this on a Dunghill at Fulham near London]" UL.\niii. "Amaranthus Blitum. Monoec: 5. and:" [JL]\''

In [36]:
print("\n\n".join(outputs))

Acer campestre L.
1 folder. Acer campestre L.

Acer pseudoplatanus L.
2 folders. 07
Folder 1. Acer Pseudo-Platanus L. "Maple. Bulls: Bulstrode Park" JL
Folder 2. Acer Pseudo-Platanus L. [TA]

'Amaranthus lividus L., Flora Europaea
1: 110 (1964)
1 folder. Amaranthus Blitum [TA].
i. Cites Ray's Syn. 1957.
ii. "Blite Amaranth. Aug. It is often found on Dunghills in the neighbourhood of London. [gather'd this on a Dunghill at Fulham near London]" UL.
iii. "Amaranthus Blitum. Monoec: 5. and:" [JL]'

Hedera helix L.
1 folder. Hedera helix L.

Asarum europaeum L.
1 folder. Asarum europaeum L.

i. "Asarum europaeum Lin: Asarabacca. Anglis. ex horto" [JL]

Impatiens noli-tangere L.
1 folder. Impatiens noli-tangere L.

[G]

Alnus glutinosa (L.) Gaertn.
1 folder. Alnus glutinosa (L.) Gaertn.

Betula nana L.
1 folder. Betula nana L.

Campanula latifolia L.
2 folders. Folder 1. Campanula latifolia L.
Folder 2. Campanula latifolia [G].

Campanula patula L.
1 folder. Campanula patula L.

Campanula ra

In [33]:
print("\n".join(cleaned_chunks))

In [34]:
tc.merge_sentences(cleaned_chunks)

NameError: name 'tc' is not defined

In [ ]:
print(cleaned_chunks[0])

Arenaria serpyllifolia L.
Folder. Arenaria Serpyllifolia [TA].
Bufonia tenuifolia L., Flora Europaea.'
1: 133 (1964)
1 folder. Bufonia Tenuifolia [TA].
[Not a British plant, but from southern and eastern Europe.]
Cerastium alpinum L.
2 folders.
Folder 1. Cerastium latifolium [G].
i. Cites Ray's Syn. 349. ii. "Cerastium alpinum. This was gathered upon Snowdon at the top of the highest Rock call'd Clogwyn y Garnedh. June" [JL]. iii. "Top of Snowdon" DL]. iv. "Dr. Solander affirms this to be the true Cerastium alpinum. I had it from Snowdon. He found it at Terra del Fuego & named it C. hirtum with a mark of Dubitation, but now thinks them both one" [JL].
Folder 2. Cerastium alpinum [TA]; Cerastium - latifolium [G]. i "Clogwyn da" [JL]. ii. *Cerast: alpin: Ben Lomond" Ut].
Cerastium arcticum Lange
2 folders.
Folder 1. Cerastium alpinum [G].
i. "Anglesey. Cerastium alpinum. Q: an Var: aut nova species?" Fig: Cerast: alp: non quadrat: Flor: Dan: The Fig: in the fl. Dan: of Cerast: alpinum ag

In [ ]:
cleaned_chunks

['Arenaria serpyllifolia L.\nfolder. Arenaria Serpyllifolia [TA].\nBufonia tenuifolia L., Flora Europaea.\'\n1: 133 (1964)\n1 folder. Bufonia Tenuifolia [TA].\n[Not a British plant, but from southern and eastern Europe.]\nCerastium alpinum L.\n2 folders.\nFolder 1. Cerastium latifolium [G].\ni. Cites Ray\'s Syn. 349. ii. "Cerastium alpinum. This was gathered upon Snowdon at the top of the highest Rock call\'d Clogwyn y Garnedh. June" [JL]. iii. "Top of Snowdon" DL]. iv. "Dr. Solander affirms this to be the true Cerastium alpinum. I had it from Snowdon. He found it at Terra del Fuego & named it C. hirtum with a mark of Dubitation, but now thinks them both one" [JL].\nFolder 2. Cerastium alpinum [TA]; Cerastium - latifolium [G]. i "Clogwyn da" [JL]. ii. *Cerast: alpin: Ben Lomond" Ut].\nCerastium arcticum Lange\n2 folders.\nFolder 1. Cerastium alpinum [G].\ni. "Anglesey. Cerastium alpinum. Q: an Var: aut nova species? Fig: Cerast: alp: non quadrat: Flor: Dan: The Fig: in the fl. Dan: of 